In [1]:
import sys
import json
import base64
import requests as rqt

In [3]:
import os
import boto3
import re
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

In [269]:
#Chargement du Modele Keras

In [4]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [5]:
import keras
from keras.models import model_from_json

Using TensorFlow backend.


In [276]:
!mkdir keras_model

mkdir: cannot create directory ‘keras_model’: File exists


In [6]:
!ls keras_model


mpg_model.json	mpg_model-weights.h5


In [7]:
import tensorflow as tf
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'mpg_model.json', 'r')
loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json,custom_objects={"GlorotUniform": tf.keras.initializers.glorot_uniform})
json_file.close()

In [8]:
#Chargement du modèle
se=keras.backend.get_session()

In [9]:
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/mpg_model-weights.h5')
print("Loaded model from disk")


Loaded model from disk


In [10]:
# conversion en type ProtBuf pour le deploiement :
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes for the next section
parent_dir='/home/ec2-user/SageMaker/'
model_version = '1'
export_dir = 'export/Servo/' + model_version


In [11]:
!ls /home/ec2-user/SageMaker/export/Servo/1


saved_model.pb	variables


In [12]:
#le dossier de destination doit contenir seulement le nv fichier
import shutil
shutil.rmtree(export_dir)


In [13]:

# on construit notre Build protocol vers le dossier export_dir
build = builder.SavedModelBuilder(export_dir)

In [14]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [15]:
from keras import backend as K

with K.get_session() as sess:
    # enregistrer les variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    build.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [13]:
!ls export/Servo/1/variables

variables.data-00000-of-00001  variables.index


In [16]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [17]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [60]:
#Créer un fichier d'entrée vide
!touch train.py

In [18]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [19]:

%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!CPU times: user 402 ms, sys: 27.6 ms, total: 430 ms
Wall time: 7min 32s


In [20]:
predictor.endpoint

'sagemaker-tensorflow-2020-07-09-16-42-30-339'

In [21]:
endpoint_name ='sagemaker-tensorflow-2020-07-09-16-42-30-339'

In [22]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)